In [55]:
!pip3 install faiss-cpu sentence-transformers transformers

  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.6 MB/s eta 0:00:00
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [sentence-transformers]ence-transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.8.36 requires accelerate==1.2.1, but you have accelerate 1.9.0 which is incompatible.
autotrain-advanced 0.8.36 requires huggingface-hub==0.27.0, but you have huggingface-hub 0.33.4 which is incompatible.
autotrain-advanced 0.8.36 requires packaging==24.2, but you have packaging 25.0 which is incompatible.
autotrain-advanced 0.8.36 requires Pillow==11.0.0, but y

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Descargar y guardar el modelo de embeddings localmente
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedder.save("modelos/all-MiniLM-L6-v2")

# Descargar y guardar el tokenizer y modelo de lenguaje
modelo_id = "microsoft/Phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
tokenizer.save_pretrained("modelos/Phi-4-mini-instruct")

modelo = AutoModelForCausalLM.from_pretrained(modelo_id)
modelo.save_pretrained("modelos/Phi-4-mini-instruct")

In [1]:
import pandas as pd

df = pd.read_csv("procesos/productos_corpus.csv",delimiter=",")
len(df)

46490

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Cargar modelo localmente
embedder = SentenceTransformer("modelos/all-MiniLM-L6-v2")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
df['CHUNK_DESCRIPCION'] = df['CHUNK_DESCRIPCION'].fillna('')
df['CHUNK_CARACTERISTICAS'] = df['CHUNK_CARACTERISTICAS'].fillna('')
df['CHUNK_OBSERVACIONES'] = df['CHUNK_OBSERVACIONES'].fillna('')
df['CHUNK_RECOMENDACIONES'] = df['CHUNK_RECOMENDACIONES'].fillna('')

In [63]:
# Convertir textos a embeddings
corpus_embeddings = embedder.encode(df['CHUNK_PRODUCTO'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_PRODUCTO.npy", corpus_embeddings)

corpus_embeddings = embedder.encode(df['CHUNK_FICHA'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_FICHA.npy", corpus_embeddings)

corpus_embeddings = embedder.encode(df['CHUNK_DESCRIPCION'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_DESCRIPCION.npy", corpus_embeddings)

corpus_embeddings = embedder.encode(df['CHUNK_CARACTERISTICAS'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_CARACTERISTICAS.npy", corpus_embeddings)

corpus_embeddings = embedder.encode(df['CHUNK_OBSERVACIONES'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_OBSERVACIONES.npy", corpus_embeddings)

corpus_embeddings = embedder.encode(df['CHUNK_RECOMENDACIONES'].tolist(), convert_to_numpy=True)
np.save("embeddings/CHUNK_RECOMENDACIONES.npy", corpus_embeddings)

In [89]:
embeddings = np.load("embeddings/CHUNK_PRODUCTO.npy")
len(embeddings)

46490

In [94]:
embeddings = np.load("embeddings/CHUNK_PRODUCTO.npy")
embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_FICHA.npy")))
embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_DESCRIPCION.npy")))
embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_CARACTERISTICAS.npy")))
embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_OBSERVACIONES.npy")))
embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_RECOMENDACIONES.npy")))
len(embeddings)

278940

In [95]:
# Crear índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
def responder_pregunta_rag(pregunta, k=10):
    # Embed la pregunta
    pregunta_emb = embedder.encode([pregunta], convert_to_numpy=True)

    # Buscar los k textos más cercanos
    distancias, indices = index.search(pregunta_emb, k)
    print(distancias)
    print(indices)

responder_pregunta_rag('necesito una refrigeradora')

[[0.5738011  0.5738011  0.5855775  0.61449397 0.6273186  0.6365832
  0.639634   0.639634   0.639634   0.639634  ]]
[[256196 256215 116950 105933 193892 112340 106684 106685 106686 106687]]


In [115]:
responder_pregunta_rag('necesito una refrigeradora')

[[0.47783175 0.48447216 0.48447216 0.48447216 0.48447216 0.48447216
  0.48447216 0.48447216 0.48447216 0.5248814 ]]
[[176543 224699 224700 224701 224702 224703 224704 227696 227697 227731]]


In [116]:
#resto = 256196 % 46490
cociente, resto = divmod(176543, 46490)
print(cociente)
print(resto)
# ['CHUNK_PRODUCTO', 'CHUNK_FICHA', 'CHUNK_DESCRIPCION', 'CHUNK_CARACTERISTICAS', 'CHUNK_OBSERVACIONES', 'CHUNK_RECOMENDACIONES']
# 46490 productos del 0 al 46489

3
37073


In [117]:
df.iloc[37073]

SKU                                                                 156424
DESCRIPCION                        OSTER FRIGOBAR 122LTS OS-PMB129BB NEGRO
DES_DIV                                                       HOGAR Y DECO
DES_AREA                                                     ELECTRO HOGAR
DES_DPTO                                                      LINEA BLANCA
DES_LIN                                                     REFRIGERADORAS
DES_MARCA                                                            OSTER
DES_PROCE                                                         NACIONAL
URL                                                                    NaN
PRECIO                                                               999.0
CHUNK_PRODUCTO           oster frigobar 122lts os-pmb129bb negro de la ...
CHUNK_FICHA              acabado liso, altura del producto 84.00 cm, an...
CHUNK_DESCRIPCION        Compartimiento refrigerado. Control de tempera...
CHUNK_CARACTERISTICAS    